In [79]:
from solana.rpc.api import Client
from solana.publickey import PublicKey
import time
import base64
from ordered_set import OrderedSet

In [96]:
client = Client("https://api.mainnet-beta.solana.com")
EVENT_QUEUE = PublicKey("FTtrtEkcJaa84FRBwp7w5fUypBwvMsbNvS3KUD1HL97c")

messages = dict()
seen = OrderedSet()

In [97]:
# Event Queue Heading Parsing
class EventQueueHeader: 
    def __init__(self, bytes):
        self.account_flags = int.from_bytes(bytes[0:8], "little")
        self.head = int.from_bytes(bytes[8:16], "little")
        self.count = int.from_bytes(bytes[16:24], "little")
        self.seq_num = int.from_bytes(bytes[24:32], "little")
    def __str__(self):
        return (
            "Account Flags: " + str(self.account_flags) + "\n" +
            "Head: " + str(self.head) + "\n" +
            "Count: " + str(self.count) + "\n" + 
            "Sequence Number: " + str(self.seq_num) + "\n" 
        )

In [98]:
class Event:
    def __init__(self, bytes):
        self.event_flags = int.from_bytes(bytes[0:1], "little")
        self.owner_slot = int.from_bytes(bytes[1:2], "little")
        self.fee_tier = int.from_bytes(bytes[2:3], "little")
        padding = []
        for i in range(5):
            padding.append(int.from_bytes(bytes[3+i:4+i], "little"))
        self._padding = padding
        self.native_quantity_released = int.from_bytes(bytes[8:16], "little")
        self.native_quantity_paid = int.from_bytes(bytes[16:24], "little")
        self.native_fee_or_rebate = int.from_bytes(bytes[24:32], "little")
        self.order_id = int.from_bytes(bytes[32:48], "little")
        self.owner = PublicKey(bytes[48:80])
        self.client_order_id = int.from_bytes(bytes[80:], "little")
    def __str__(self):
        return (
            "Event Flags: " + str(self.event_flags) + "\n" + 
            "Owner Slot: " + str(self.owner_slot) + "\n" + 
            "Fee Tier: " + str(self.fee_tier) + "\n" + 
            "Padding: " + ', '.join(str(item) for item in self._padding) + "\n" + 
            "Native Quantity Released: " + str(self.native_quantity_released) + "\n" +
            "Native Quantity Paid: " + str(self.native_quantity_paid) + "\n" + 
            "Native Fee or Rebate: " + str(self.native_fee_or_rebate) + "\n" + 
            "Order ID: " + str(self.native_fee_or_rebate) + "\n" + 
            "Owner: " + ', '.join(str(item) for item in self.owner) + "\n" + 
            "Client Order ID: " + str(self.client_order_id) + "\n"
        )

In [99]:
class EventAccount:
    def __init__(self, bytes):
        bytes_without_padding = bytes[5:-7]
        header_bytes = bytes_without_padding[0:32]
        events_bytes = bytes_without_padding[32:]
        self.event_queue_header = EventQueueHeader(header_bytes)
        self.events = []
        for i in range(self.event_queue_header.count):
            starting_index, ending_index = \
                self.event_to_byte_index(self.event_queue_header.head + i)
            self.events.append(Event(
                events_bytes[starting_index:ending_index]
            ))
            
    def event_to_byte_index(self, event_index):
        starting_index = event_index * 88
        ending_index = starting_index + 88
        return (starting_index, ending_index)
    
    def print_overview(self):
        print(self.event_queue_header)
        
    def print_events(self):
        for i in range(self.event_queue_header.count):
            print("------------ Event Number", i+1, "----------")
            print(self.events[i])

In [100]:
def process_event(event, wallet_to_volume):
    if event.owner not in wallet_to_volume:
        wallet_to_volume[event.owner] = 0
    if event.event_flags % 2 == 1:
        wallet_to_volume[event.owner] += event.native_quantity_paid
        wallet_to_volume[event.owner] += event.native_quantity_released

In [101]:
for idx in range(200):
    account_info = client.get_account_info(EVENT_QUEUE)
    account_data_raw = base64.b64decode(account_info['result']['value']['data'][0])
    assert len(account_data_raw) == 262156
    if account_data_raw not in seen:
        messages[idx] = account_data_raw
        seen.add(account_data_raw)
    time.sleep(2)

In [102]:
parsed_accounts = []
for raw_account in seen:
    parsed_accounts.append(EventAccount(raw_account))

In [107]:
len(parsed_accounts)
for parsed_account in parsed_accounts:
    parsed_account.print_overview()

Account Flags: 17
Head: 236
Count: 1
Sequence Number: 339729

Account Flags: 17
Head: 238
Count: 0
Sequence Number: 339730

Account Flags: 17
Head: 239
Count: 0
Sequence Number: 339731

Account Flags: 17
Head: 241
Count: 0
Sequence Number: 339733

Account Flags: 17
Head: 241
Count: 1
Sequence Number: 339734

Account Flags: 17
Head: 243
Count: 0
Sequence Number: 339735

Account Flags: 17
Head: 243
Count: 1
Sequence Number: 339736

Account Flags: 17
Head: 244
Count: 0
Sequence Number: 339736

Account Flags: 17
Head: 244
Count: 1
Sequence Number: 339737

Account Flags: 17
Head: 246
Count: 1
Sequence Number: 339739

Account Flags: 17
Head: 247
Count: 0
Sequence Number: 339739

Account Flags: 17
Head: 248
Count: 0
Sequence Number: 339740

Account Flags: 17
Head: 249
Count: 0
Sequence Number: 339741

Account Flags: 17
Head: 249
Count: 1
Sequence Number: 339742

Account Flags: 17
Head: 249
Count: 2
Sequence Number: 339743

Account Flags: 17
Head: 251
Count: 0
Sequence Number: 339743



In [110]:
num_parsed_accounts = len(parsed_accounts)
wallet_to_volume = {}
for i in range(num_parsed_accounts - 1):
    current_head = parsed_accounts[i].event_queue_header.head
    next_head = parsed_accounts[i+1].event_queue_header.head
    if current_head < next_head:
        difference = next_head - current_head 
    elif current_head == next_head:
        continue    
    else:
        difference = next_head + 2978 - current_head
    print("The current head, next head, and difference are", current_head, next_head, difference)
    
    current_count = parsed_accounts[i].event_queue_header.count
    for j in range(min(difference, current_count)):
        process_event(parsed_accounts[i].events[j], wallet_to_volume)
print(wallet_to_volume)

The current head, next head, and difference are 236 238 2
The current head, next head, and difference are 238 239 1
The current head, next head, and difference are 239 241 2
The current head, next head, and difference are 241 243 2
The current head, next head, and difference are 243 244 1
The current head, next head, and difference are 244 246 2
The current head, next head, and difference are 246 247 1
The current head, next head, and difference are 247 248 1
The current head, next head, and difference are 248 249 1
The current head, next head, and difference are 249 251 2
{H19Y4fn2W5baqsXrnJcMwmJkcHVTVyiUAhAKMok5W2Zm: 0}


Next Steps
1. Look for better examples on SolScan 

In [116]:
class OpenOrdersAccount: 
    def __init__(self, bytes):
        # Remove "serum" and "padding"
        bytes = bytes[5:-7]
        self.account_flags = int.from_bytes(bytes[0:8], "little")
        self.market = []
        for i in range(32):
            self.market.append(int.from_bytes(bytes[8+i:9+i], "little"))
        self.owner = PublicKey(bytes[48:72])
        self.native_base_free = int.from_bytes(bytes[72:80], "little")
        self.native_base_total = int.from_bytes(bytes[80:88], "little")
        self.native_quote_free = int.from_bytes(bytes[88:96], "little")
        self.native_quote_total = int.from_bytes(bytes[96:104], "little")
        self.free_slot_bits = int.from_bytes(bytes[104:120], "little")
        self.is_bid_bits = int.from_bytes(bytes[120:136], "little")
        self.orders = []
        for i in range(128):
            starting_index = 136 + i*16
            ending_index = 136 + (i + 1) * 16
            self.orders.append(
                int.from_bytes(bytes[starting_index:ending_index], "little")
            )
        self.client_order_ids = []
        for i in range(128):
            starting_index = 2184 + i*8
            ending_index = 2184 + (i + 1) * 8
            self.orders.append(
                int.from_bytes(bytes[starting_index:ending_index], "little")
            )
        self.referrer_rebates_accrued = int.from_bytes(bytes[3208:3216], "little")
    def get_owner(self):
        return self.owner

In [119]:
open_orders = PublicKey("H19Y4fn2W5baqsXrnJcMwmJkcHVTVyiUAhAKMok5W2Zm")
account_info = client.get_account_info(open_orders)
account_data_raw = base64.b64decode(account_info['result']['value']['data'][0])
open_orders_account = OpenOrdersAccount(account_data_raw)
open_orders_account.get_owner()

11111111Pd7FWUZjgVJZbSp4UwXNge9UcSvJYpEUd